## 0.导入库

In [1]:
# 加入下面2行，可以使py代码文件中的修改即时生效
%load_ext autoreload
%autoreload 2

In [2]:
# 如果python2使用print，也得加上括号
from __future__ import print_function

In [3]:
# 导入keras
import keras

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# 导入keras库的这4种层：全连接层Dense，2维卷积层Conv2D，批量归一化层BatchNormalization，激活层Activation
from keras.layers import Dense, Conv2D, BatchNormalization, Activation

In [5]:
# 导入keras库的这3种层：平均2维池化层AveratePooling2D, 输入层Input，激活层Activation
from keras.layers import AveragePooling2D, Input, Flatten

In [6]:
# 导入keras库的优化器：Adam优化器
from keras.optimizers import Adam

In [7]:
# 导入keras库的回传函数：模型检查点ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [8]:
# 导入keras库的回传函数：学习率递减ReduceLROnPlateau
from keras.callbacks import ReduceLROnPlateau

In [9]:
# 导入keras库的图片处理函数：图片数据生成器
from keras.preprocessing.image import ImageDataGenerator

In [10]:
# 导入keras库的正则化函数：L2正则化
from keras.regularizers import l2

In [11]:
# 导入keras库的后端：backend中文叫做后端，取别名为K
from keras import backend as K

In [12]:
# 导入keras库的模型函数：Model
from keras.models import Model

In [13]:
# 导入keras库的数据集类：cifar10
from keras.datasets import cifar10

In [14]:
# 导入必需的常用库
import numpy as np
import os

## 1.数据准备

### 1.1 加载训练集

In [15]:
## 加载数据集cifar10里面的训练集
from keras.datasets.cifar import load_batch

def load_train_dataset(dirPath='../resources/cifar-10-batches-py/'):
    train_sample_quantity = 50000
    image_width = 32
    image_height = 32
    channel_quantity = 3
    train_X = np.zeros((train_sample_quantity, channel_quantity, image_width, image_height),
                       dtype='uint8')
    train_y = np.zeros((train_sample_quantity, ),
                       dtype='uint8')
    for i in range(1, 6):
        fileName = 'data_batch_%d' %i
        filePath = os.path.join(dirPath, fileName)
        startIndex = (i - 1) * 10000
        endIndex = i * 10000
        train_X[startIndex:endIndex, :, :, :], train_y[startIndex:endIndex] = load_batch(filePath)
    print('train_X矩阵转置前：', train_X.shape)
    # 从官网上下载的数据集的4个维度为样本个数n、通道数c、宽度w、高度h
    # Keras基于Tensorflow，数据的维度顺序要求：样本个数n、宽度w、高度h、通道数c，所以使用np.transpose完成矩阵转置
    train_X = train_X.transpose(0, 2, 3, 1)
    print('train_X矩阵转置后：', train_X.shape)
    return train_X, train_y

dirPath = '../resources/cifar-10-batches-py/'
train_X, train_y = load_train_dataset() 

train_X矩阵转置前： (50000, 3, 32, 32)
train_X矩阵转置后： (50000, 32, 32, 3)


### 1.2 加载测试集

In [16]:
# 加载数据集cifar10里面的测试集
def load_test_dataset(dirPath='../resources/cifar-10-batches-py/'):
    fileName = 'test_batch'
    filePath = os.path.join(dirPath, fileName)
    test_X, test_y = load_batch(filePath)
    print('test_X矩阵转置前：', test_X.shape)
    test_X = test_X.transpose(0, 2, 3, 1)
    print('test_X矩阵转置后：', test_X.shape)
    return test_X, test_y

dirPath = '../resources/cifar-10-batches-py/'
test_X, test_y = load_test_dataset() 

test_X矩阵转置前： (10000, 3, 32, 32)
test_X矩阵转置后： (10000, 32, 32, 3)


### 1.3 对类别做One-Hot编码

In [17]:
# 对类别ID做One-Hot编码
from keras.utils import to_categorical

class_quantity = 10
train_Y = to_categorical(train_y, class_quantity)
test_Y = to_categorical(test_y, class_quantity)

In [22]:
# 设定学习率
def get_learningRate(epoch):
    learningRate = 1e-3
    if epoch > 180:
        learningRate *= 0.5e-3
    elif epoch > 160:
        learningRate *= 1e-3
    elif epoch > 120:
        learningRate *= 1e-2
    elif epoch > 80:
        learningRate *= 1e-1
    print('learning rate:', learningRate)
    return learningRate

In [20]:
# 定义resnet层
def resnet_layer(inputs,
        filter_quantity=16,
        filter_size=3,
        strides=1,
        activation='relu',
        batch_normalization=True,
        conv_first=True):
    layer_convolution = Conv2D(filter_quantity,
        kernel_size=filter_size,
        strides=strides,
        padding='same',
        kernel_initializer='he_normal',
        kernel_regularizer=l2(1e-4))
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [21]:
# 获取resnet模型
def get_resnetModel_v2(input_shape, depth, class_quantity=10):
    if (depth - 2) % 9 !=0:
        raise ValueError('depth should be 9n+2(eg 56 or 110)')
    in_filter_quantity = 16
    res_block_quantity = 16
    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs,
        filter_quantity=in_filter_quantity,
        conv_first=True)
    for stage in range(3):
        for res_block in range(res_block_quantity):
            activation = 'relu'
            batch_normalization = True
            strides =1
            if stage == 0:
                out_filter_quantity = in_filter_quantity * 4
                if res_block == 0:
                    activation=None
                    batch_normalization = False
            else:
                out_filter_quantity = in_filter_quantity * 2
                if res_block == 0:
                    strides = 2
            y = resnet_layer(inputs=x,
                filter_quantity=in_filter_quantity,
                filter_size=1,
                strides=strides,
                activation=activation,
                batch_normalization=batch_normalization,
                conv_first=False)
            y = resnet_layer(inputs=y,
                filter_quantity=in_filter_quantity,
                conv_first=False)
            y = resnet_layer(inputs=y,
                filter_quantity=out_filter_quantity,
                filter_size=1,
                conv_first=False)
            if res_block == 0:
                x = resnet_layer(inputs=x,
                    filter_quantity=out_filter_quantity,
                    filter_size=1,
                    strides=strides,
                    activation=None,
                    batch_normalization=False)
            x = keras.layers.add([x, y])
        in_filter_quantity = out_filter_quantity
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    x = Flatten(x)
    outputs = Dense(num_classes,
        activation='softmax',
        kernel_initializer='he_normal')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [24]:
input_shape = train_x.shape[1:]
resnetModel_v2 = get_resnetModel_v2(input_shape=input_shape,
    depth=depth)
resnetModel_v2.compile(loss='categorical_crossentropy',
    optimizer=Adam(lr=get_learningRate(0)),
    metrics=['accuracty'])

NameError: name 'conv' is not defined